# 1. Source

https://platform.stratascratch.com/coding/2028-new-and-existing-users?tabname=question&code_type=1

# 2. Description of the table

fact_events table:

In [ ]:
id:          int 
time_id:     datetime
user_id:     varchar
customer_id: varchar
client_id:   varchar
event_type:  varchar
event_id:    int

![table](table_.png)

# 3. The challenge

Calculate the share of new and existing users for each month in the table. 
Output the month, share of new users, and share of existing users as a ratio.

New users are defined as users who started using services in the current month (there is no usage history in previous months). Existing users are users who used services in current month, but they also used services in any previous month.
Assume that the dates are all from the year 2020.

# 4. Postgres Code

In [ ]:
with cte as(
select
    date_part('month' , time_id)  as month,
    count( distinct user_id)
from fact_events
group by 1),

month2_users as(
select
    distinct user_id
from fact_events
where date_part('month' , time_id) = 2),

month3_users as(
select 
    distinct user_id
from fact_events
where date_part('month' , time_id) = 3
and user_id not in(select * from month2_users)
),

month4_users as(
select 
    distinct user_id
from fact_events
where date_part('month' , time_id) = 4
and user_id not in(select * from month2_users)
and user_id not in(select * from month3_users)
),

month3_and_2_users as(
select
    user_id
from month2_users
union
select 
    user_id
from month3_users
),

month4_and_3_users as (
select
    user_id
from month3_users
union
select 
    user_id
from month4_users
),

new_user_count as(
select
    2 as month,
    count(*)
from (select * from month2_users) as sub1
union
select
    3 as month,
    count(*)
from (select * from month3_users) as sub1
union
select
    4 as month,
    count(*)
from (select * from month4_users) as sub1
order by month),

share_of_new_users_table as(
select 
    nuc.month,
    nuc.count as new_users,
    c.count as month_user_count,
    nuc.count::numeric / c.count as share_new_users
from new_user_count as nuc 
join cte as c 
on nuc.month = c.month),

months_2_and_3 as(
select 
    count(user_id)
from (
select 
    distinct user_id
from fact_events
where date_part('month' , time_id) = 3
and user_id in(select * from month2_users)
) as s),


months_2_and_3_4 as(
select 
    count(distinct user_id)
from (
select 
    distinct user_id
from fact_events
where date_part('month' , time_id) = 4
and user_id in(select * from month3_and_2_users)
) as s),

existing_users_table as(
select 
    2 as month,
    0 as existing_users
union
select
    3 as month,
    count as existing_users
from 
    months_2_and_3
union
select
    4 as month,
    count as existing_users
from months_2_and_3_4
order by month),

final as(
select
    n.month,
    n.new_users,
    n.month_user_count,
    n.share_new_users,
    e.existing_users,
    e.existing_users::numeric / n.month_user_count as share_existing_users
from share_of_new_users_table n 
join existing_users_table e 
on n.month =  e.month)

select  
    month,share_new_users,share_existing_users
from final
;

# 5.Output / Solution

![hey](solution.png)